# <font color=#e41b52> **CoxPH univariate**
_______

In [1]:
import os
import sys
sys.path.insert(0, '/home/labs/hornsteinlab/hadarkl/Hadar/primals/code')

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import KFold
from lifelines import CoxPHFitter, KaplanMeierFitter, statistics

from statsmodels.stats.multitest import multipletests
from scipy.stats import norm, zscore
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
#---------------------------------------------------------------------------
#loaders
%load_ext autoreload
%autoreload 2


from configs.constants import IMMUNE_PROTEINS, PROCESSED_PROTEOMICS_DATA, KERATIN,CLINICAL_DATA_DISCOVERY,MAIN_DIR, seed,MODEL_DIR
from configs.proteins_lists import SELECTED_FEATURES_COX_REP
from utils.utils_protein_processing import scale_data
from utils.survival_analysis_utils import dichotomize_proteins_by_median, cox_proportional_hazard_model


## 📎 <font color=#df208f>Load Data 
___

### <font color = "pink" > Data:
- log after combat
- missing values 


In [2]:
log_after_combat_discovery = pd.read_csv('/home/labs/hornsteinlab/hadarkl/Hadar/primals/code/data/raw_data/proteins/DIANN/COMBAT/log_after_combat_discovery.csv')
#log_after_combat_discovery  = log_after_combat_discovery .set_index('sample number')
log_after_combat_replication = pd.read_csv('/home/labs/hornsteinlab/hadarkl/Hadar/primals/code/data/raw_data/proteins/DIANN/COMBAT/log_after_combat_replication.csv')
#log_after_combat_replicatio = log_after_combat_replicatio.set_index('sample number')



In [3]:
print(f'Number of Na discovery : {log_after_combat_discovery.isnull().sum().sum()}') ,print(f'Number of Na Replication : {log_after_combat_replication.isnull().sum().sum()}')


Number of Na discovery : 0
Number of Na Replication : 0


(None, None)

### <font color = "pink" >Data:
- log before combat 

- no missing values


In [7]:
log_before_combat_discovery = pd.read_csv('/home/labs/hornsteinlab/hadarkl/Hadar/primals/code/data/raw_data/proteins/DIANN/COMBAT/log_before_combat_discovery.csv')
log_before_combat_discovery = log_before_combat_discovery.set_index('sample number')
log_before_combat_replication = pd.read_csv('/home/labs/hornsteinlab/hadarkl/Hadar/primals/code/data/raw_data/proteins/DIANN/COMBAT/log_before_combat_replication.csv')
log_before_combat_replication = log_before_combat_replication.set_index('sample number')

In [50]:
print(f'Number of Na discovery : {log_before_combat_discovery.isnull().sum().sum()}') ,print(f'Number of Na Replication : {log_before_combat_replication.isnull().sum().sum()}')


Number of Na discovery : 0
Number of Na Replication : 0


(None, None)

### <font color = "pink" > In my current analysis I Decided to continue with this data :
- log before combat 
- low corr data(remove r < 0.7)
- no missing values
- clinical data (Survival_from_onset (months), Status dead=1, Age Onset ,ALSFRS score,ALSFRS_slope)


In [2]:

log_before_combat_discovery_low_collinearity = pd.read_csv('/home/labs/hornsteinlab/hadarkl/Hadar/primals/code/data/raw_data/proteins/DIANN/COMBAT/log_before_combat_discovery_low_collinearity.csv')

log_before_combat_discovery_low_collinearity= log_before_combat_discovery_low_collinearity.rename(columns={'sample_number': 'sample number'})
log_before_combat_discovery_low_collinearity = log_before_combat_discovery_low_collinearity.set_index('sample number')
log_before_combat_replication_low_collinearity = pd.read_csv('/home/labs/hornsteinlab/hadarkl/Hadar/primals/code/data/raw_data/proteins/DIANN/COMBAT/log_before_combat_replication_low_collinearity.csv')
log_before_combat_replication_low_collinearity= log_before_combat_replication_low_collinearity.rename(columns={'sample_number': 'sample number'})
log_before_combat_replication_low_collinearity = log_before_combat_replication_low_collinearity.set_index('sample number')


In [9]:
log_before_combat_replication_low_collinearity

,A1BG,ABCA1,ABCE1,ABHD14B,ABI3BP,ACACA,ACAN,ACAT2,ACE,ACO1,...,ZG16,ZG16B,ZNF518A,ZNF534,ZNF550,ZNF639,ZNF705B.ZNF705D,ZNF728,ZNF839,ZYX
sample number,,,,,,,,,,,,,,,,,,,,,
705,32.767019,14.617035,22.405854,15.767252,23.100818,27.276984,19.622691,17.587580,22.884085,19.206655,...,22.349299,19.261815,22.150588,22.544939,21.176072,24.098878,22.853966,20.079326,19.193004,20.008833
712,31.672679,16.129750,22.435973,15.985590,23.120916,24.807393,21.594033,19.656477,23.465324,18.663970,...,23.085535,23.143255,22.488943,21.583958,25.090641,23.752618,21.910224,18.013549,17.048372,20.984729
717,32.416262,20.135895,22.507210,15.628147,23.364433,25.521301,20.875044,14.841177,23.367109,19.634724,...,22.884191,17.605121,23.176669,22.205856,21.176072,23.815770,23.713183,20.161759,18.991899,18.837074
735,32.280831,14.617035,22.722695,15.262364,23.612653,25.866303,20.695015,19.855753,23.571384,18.991296,...,22.439549,17.632425,22.921454,22.046361,25.717360,23.668922,23.678001,19.979316,17.048372,19.772410
746,32.045909,14.617035,21.425236,16.801405,23.012890,26.479674,20.862343,19.665163,22.476821,21.279045,...,23.040817,17.516713,21.460545,21.432838,23.706523,24.052827,21.389616,18.013549,17.048372,21.086846
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1899,32.043843,17.306311,21.947075,13.032740,23.065568,23.999447,20.381529,18.297914,22.506884,20.141768,...,24.334272,18.333983,20.001861,21.307342,24.749209,25.486360,18.860192,18.013549,18.078638,21.420257
1900,32.039400,18.881314,23.250763,13.032740,22.752819,24.391439,21.002702,18.088029,22.747360,20.477333,...,23.684111,16.262618,21.253571,23.300752,24.744619,25.244121,22.532217,20.280551,19.855612,19.034876
1912,32.171472,20.574169,23.560576,13.032740,23.594394,25.582546,19.534464,14.841177,23.218307,18.475932,...,21.349646,19.026571,21.527942,22.291282,25.005046,25.297858,21.907846,18.951901,18.178938,19.876182


In [3]:
print(f'Number of Na discovery : {log_before_combat_discovery_low_collinearity.isnull().sum().sum()}') ,print(f'Number of Na Replication : {log_before_combat_replication_low_collinearity.isnull().sum().sum()}')


Number of Na discovery : 0
Number of Na Replication : 0


(None, None)

In [3]:
log_before_combat_replication_low_collinearity_scaled = scale_data(log_before_combat_replication_low_collinearity, scaling="zscore").reset_index('sample number')

log_before_combat_discovery_low_collinearity_scaled = scale_data(log_before_combat_discovery_low_collinearity, scaling="zscore").reset_index('sample number')

Z-score normalization applied.
Z-score normalization applied.


In [26]:
# df_combat_dis.to_csv('log_before_combat_discovery.csv', index= True)
# df_combat_rep.to_csv('log_before_combat_replication.csv', index=True)

### <font color = "pink" > Clinical data

In [4]:
#clinical data
clinical_discovery = pd.read_csv('/home/labs/hornsteinlab/Collaboration/PRIMALS/Serum/Hadar/primals/code/data/raw_data/clinical_data/clinical_data_discovery.csv')
clinical_discovery = clinical_discovery[['sample number','Survival_from_onset (months)','Status dead=1']]
#clinical_dis =clinical_dis.set_index('sample number')
clinical_replication = pd.read_csv('/home/labs/hornsteinlab/Collaboration/PRIMALS/Serum/Hadar/primals/code/data/raw_data/clinical_data/clinical_data_replication.csv')
clinical_replication= clinical_replication[['sample number','Survival_from_onset (months)','Status dead=1']]
#clinical_rep = clinical_rep.set_index('sample number')

_-__-__-__-__-__-__-__-__-__-__-__-__-_>



### Merge protein data with clinical data

In [5]:
df_discovery_merge = pd.merge(log_before_combat_discovery_low_collinearity_scaled, clinical_discovery, on='sample number', how='left')
df_discovery_merge = df_discovery_merge.set_index('sample number')
df_replication_merge = pd.merge(log_before_combat_replication_low_collinearity_scaled, clinical_replication, on='sample number', how='left')
df_replication_merge = df_replication_merge.set_index('sample number')

In [6]:
df_discovery_merge

,A1BG,ABCA1,ABCE1,ABHD14B,ABI3BP,ACACA,ACAN,ACAT2,ACE,ACO1,...,ZNF518A,ZNF534,ZNF550,ZNF639,ZNF705B.ZNF705D,ZNF728,ZNF839,ZYX,Survival_from_onset (months),Status dead=1
sample number,,,,,,,,,,,,,,,,,,,,,
711,1.252098,-0.958143,0.748834,-0.349957,0.826279,0.561998,1.208108,0.237486,0.012301,0.564561,...,-0.382598,1.326908,-1.031698,1.180792,0.454344,-1.550532,-0.479163,-1.466864,21,1
743,0.634843,-0.958143,0.693342,0.540603,1.037509,0.647283,0.592872,0.679847,1.787424,0.488853,...,0.302088,0.470459,0.462548,-0.644853,0.116095,-1.929910,0.836075,-0.077404,44,1
744,0.709325,0.877534,1.205605,0.825283,-0.434558,1.678959,0.545695,1.084028,0.395276,1.179849,...,-0.449205,0.641015,-0.361956,-0.359838,0.783894,-1.833087,0.589344,0.170805,62,1
747,-0.018939,-0.958143,1.003098,0.200843,0.330903,0.927188,0.051449,0.267404,1.199327,0.069852,...,-0.601829,1.363850,-1.370161,1.616894,-0.398571,-1.641528,0.424275,-1.195902,33,1
749,1.113657,-0.958143,1.479285,-0.821672,0.807090,0.828731,0.648740,0.660717,0.396426,1.316190,...,-0.008856,0.944451,0.050117,-0.114192,-0.625234,-1.106872,-0.629070,-1.745603,49,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1884,-0.043852,-0.958143,-0.549563,-1.641779,0.777862,1.160038,-0.041460,-1.938069,0.325318,0.426078,...,0.968153,-1.692564,1.307037,0.578152,-1.518262,0.617315,-2.304026,-1.318064,73,1
1932,-0.395311,-0.958143,0.135010,0.926345,-1.469599,-0.465914,0.273595,-1.938069,-0.554875,-0.908446,...,-0.483812,0.667801,0.039973,2.033042,0.870081,0.953537,-0.049856,1.822968,44,0
2053,-3.365165,-0.958143,-0.801404,0.243924,-0.595623,-2.198694,-0.140481,0.346868,0.355025,0.117790,...,0.908583,-0.924028,1.833290,-1.275952,-1.518262,0.115394,-2.304026,0.590017,61,1


_-__-__-__-__-__-__-__-__-__-__-_

### dichotomizaton

In [ ]:

def dichotomize_protein(df, protein_column, threshold):
    """
    Convert a selected protein column to a dichotomous (binary) format based on a threshold.

    :param df: DataFrame containing protein expression data.
    :param protein_column: Name of the protein column to dichotomize.
    :param threshold: Numeric threshold to classify values as 0 or 1.
    :return: A new DataFrame with a binary column.
    """
    if protein_column not in df.columns:
        raise ValueError(f"Column '{protein_column}' not found in DataFrame.")
    
    df_binary = df.copy()
    
    # Apply dichotomization: 1 if above threshold, else 0
    df_binary[f"{protein_column}_dichotomized"] = (df_binary[protein_column] > threshold).astype(int)
    
    return df_binary[[protein_column, f"{protein_column}_dichotomized"]]


result_rep = dichotomize_protein(df_combat_rep, "LRG1", 30.5641817962889)  # Apply threshold on "LRG1" protein
result_rep
result_dis = dichotomize_protein(df_combat_dis, "LRG1", 30.5641817962889)  # Apply threshold on "LRG1" protein
result_dis


# <font color = #37aec8 >  📎 Models
______

## <font color="lightblue"/> cox univariant model </font>

In [ ]:
# ##nancy code
# cox_p = {} 
# protein_names = [col for col in data.columns if col not in ['Status dead=1',  'Survival_from_onset (months)']]
# for protein_name in protein_names:
    
#     cox_data = data.dropna(subset=[protein_name])
#     cph = cox_proportional_hazard_model(cox_data[[protein_name, 'Status dead=1',  'Survival_from_onset (months)']],
#                                         'Survival_from_onset (months)',
#                                         'Status dead=1',
#                                         protein_name,
#                                         strata=None,
#                                         covariate_groups=None,
#                                         show_plot=False)
    
#     p_value = cph.summary.loc[protein_name]['p']
#     cox_p[protein_name] = p_value


## <font color="lightblue"/> Without cross validation


In [7]:
data  = df_discovery_merge
cox_p = {}
concordance_index = {}
hazard_ratios = {}
ci_lower = {}
ci_upper = {}

# List of protein names to analyze
protein_names = [col for col in data.columns if col not in ['Status dead=1', 'Survival_from_onset (months)']]

for protein_name in protein_names:
    
    # Drop NA values for the current protein
    cox_data = data.dropna(subset=[protein_name])

    # Run the Cox proportional hazards model on the entire dataset (no CV)
    cph = CoxPHFitter()
    cph.fit(cox_data[[protein_name, 'Status dead=1', 'Survival_from_onset (months)']],
            duration_col='Survival_from_onset (months)',
            event_col='Status dead=1')
    
    # Extract results from the model
    p_value = cph.summary.loc[protein_name, 'p']
    c_index = cph.concordance_index_
    hazard_ratio = cph.hazard_ratios_[protein_name]
    CI_L = cph.summary.loc[protein_name, "exp(coef) lower 95%"]
    CI_U = cph.summary.loc[protein_name, "exp(coef) upper 95%"]

    # Store results
    cox_p[protein_name] = p_value
    concordance_index[protein_name] = c_index
    hazard_ratios[protein_name] = hazard_ratio
    ci_lower[protein_name] = CI_L
    ci_upper[protein_name] = CI_U

# Create a DataFrame with the results
results = pd.DataFrame({
    'p_value': pd.Series(cox_p),
    'concordance_index': pd.Series(concordance_index),
    'hazard_ratio': pd.Series(hazard_ratios),
    'CI_lower': pd.Series(ci_lower),
    'CI_upper': pd.Series(ci_upper)
})
results


,p_value,concordance_index,hazard_ratio,CI_lower,CI_upper
A1BG,0.472479,0.491119,1.063061,0.899717,1.256061
ABCA1,0.460486,0.518856,1.062856,0.904001,1.249625
ABCE1,0.101616,0.558324,1.152392,0.972411,1.365686
ABHD14B,0.029721,0.571812,1.213312,1.019201,1.444392
ABI3BP,0.003657,0.568664,0.792926,0.678073,0.927232
...,...,...,...,...,...
ZNF639,0.509475,0.534862,1.060955,0.889910,1.264876
ZNF705B.ZNF705D,0.326197,0.526843,0.927277,0.797535,1.078125
ZNF728,0.026237,0.571514,0.850160,0.736774,0.980995
ZNF839,0.057746,0.528997,1.177771,0.994639,1.394620


###  Results

**Filter significant proteins - discovery**




In [ ]:

significant_proteins_dis = results[results['p_value'] <= 0.05]
significant_proteins_dis

**Filter significant proteins - replication**




In [ ]:

significant_proteins_rep = results[results['p_value'] <= 0.05]
significant_proteins_rep

## <font color="lightblue"/> With cross validation


In [20]:
data = df_replication_merge[discovery_after_fdr + ['Survival_from_onset (months)', 'Status dead=1']]
#data  = df_discovery_merge
cox_p = {}
concordance_index = {}
hazard_ratios = {}
ci_lower = {}
ci_upper = {}

# List of protein names to analyze
protein_names = [col for col in data.columns if col not in ['Status dead=1', 'Survival_from_onset (months)']]

# Initialize KFold cross-validation
kf = KFold(n_splits=12)

for protein_name in protein_names:
    
    # Drop NA values for the current protein
    cox_data = data.dropna(subset=[protein_name])
    
    # Prepare to store cross-validation results
    p_values = []
    c_indices = []
    hazard_ratios_cv = []
    ci_lowers = []
    ci_uppers = []

    for train_index, test_index in kf.split(cox_data):
        train_data = cox_data.iloc[train_index]
        test_data = cox_data.iloc[test_index]

        # Run the Cox proportional hazards model
        cph = cox_proportional_hazard_model(train_data[[protein_name, 'Status dead=1', 'Survival_from_onset (months)']],
                                            'Survival_from_onset (months)',
                                            'Status dead=1',
                                            protein_name,
                                            strata=None,
                                            covariate_groups=None,
                                            show_plot=False)

        # Extract results from the model trained on this fold
        p_value = cph.summary.loc[protein_name]['p']
        c_index = cph.concordance_index_
        hazard_ratio = cph.hazard_ratios_[protein_name]
        #coef = cph.params_[protein_name]
      
        CI_L = cph.summary.loc[protein_name]["exp(coef) lower 95%"]
        CI_U = cph.summary.loc[protein_name]["exp(coef) upper 95%"]

        #Store results for this fold
        p_values.append(p_value)
        c_indices.append(c_index)
        hazard_ratios_cv.append(hazard_ratio)
        ci_lowers.append(CI_L)
        ci_uppers.append(CI_U)
        

    # Calculate mean of the cross-validation results for each metric
    cox_p[protein_name] = np.mean(p_values)
    concordance_index[protein_name] = np.mean(c_indices)
    hazard_ratios[protein_name] = np.mean(hazard_ratios_cv)
    #coef[protein_name] =np.mean(coef)
    ci_lower[protein_name] = np.mean(ci_lowers)
    ci_upper[protein_name] = np.mean(ci_uppers)

# Create a DataFrame with the results
results_cv = pd.DataFrame({
    'p_value': pd.Series(cox_p),
    'concordance_index': pd.Series(concordance_index),
    'hazard_ratio': pd.Series(hazard_ratios),
    'CI_lower': pd.Series(ci_lower),
    'CI_upper': pd.Series(ci_upper)
})





 Concordance index of the model 0.5622303882409331


 Concordance index of the model 0.561256293454807


 Concordance index of the model 0.5498739363378506


 Concordance index of the model 0.5521168800565549


 Concordance index of the model 0.5509657075285771


 Concordance index of the model 0.5597617368132298


 Concordance index of the model 0.5495968057621545


 Concordance index of the model 0.5768929912390488


 Concordance index of the model 0.5567584529693261


 Concordance index of the model 0.5488993710691824


 Concordance index of the model 0.5622205088666152


 Concordance index of the model 0.566215599817546


 Concordance index of the model 0.5221281354849018


 Concordance index of the model 0.5211380164628786


 Concordance index of the model 0.5373463599117554


 Concordance index of the model 0.5342863875579295


 Concordance index of the model 0.5235317303902247


 Concordance index of the model 0.5350732816051414


 Concordance index of the model 0.534956548970

### <font color =#16a5e9 >Results :
_ _ _ _

### Discovery

**Sort the DataFrame by p-values in decreasing order**

In [10]:

#Sort the DataFrame by p-values in **decreasing order**
discovery = results_cv.sort_values(by='p_value', ascending=True)
discovery

,p_value,concordance_index,hazard_ratio,CI_lower,CI_upper
CILP2,4.362187e-08,0.650983,0.591399,0.493154,0.709223
REG1A,7.951281e-07,0.629534,1.513684,1.288619,1.778114
GRN,8.033855e-07,0.650086,1.541034,1.306702,1.817437
MRC1,2.843527e-06,0.640314,1.529697,1.292062,1.811114
IGFBP2,4.543928e-06,0.636588,1.504184,1.269023,1.782994
...,...,...,...,...,...
ST13,8.832421e-01,0.501142,0.988869,0.834581,1.171691
CYFIP1,8.835076e-01,0.501477,1.006589,0.847142,1.196063
TIMP3,8.844964e-01,0.499407,1.000409,0.845956,1.183067
PMEL,8.991300e-01,0.504902,0.999569,0.846083,1.180907


**Apply FDR correction (Benjamini-Hochberg method)**

In [11]:
# Step 2: Apply FDR correction (Benjamini-Hochberg method) and store in a new column
discovery['FDR_adjusted_pval'] = multipletests(discovery['p_value'], method='fdr_bh')[1]

**Filter significant proteins(<0.05)**

In [12]:
significant_proteins_discovery_fdr = discovery[discovery['FDR_adjusted_pval'] <= 0.05]
significant_proteins_discovery_fdr

,p_value,concordance_index,hazard_ratio,CI_lower,CI_upper,FDR_adjusted_pval
CILP2,4.362187e-08,0.650983,0.591399,0.493154,0.709223,0.000055
REG1A,7.951281e-07,0.629534,1.513684,1.288619,1.778114,0.000336
GRN,8.033855e-07,0.650086,1.541034,1.306702,1.817437,0.000336
MRC1,2.843527e-06,0.640314,1.529697,1.292062,1.811114,0.000891
IGFBP2,4.543928e-06,0.636588,1.504184,1.269023,1.782994,0.001140
ADAM15,7.326655e-06,0.637693,1.475305,1.254887,1.734510,0.001531
EPHB4,3.174436e-05,0.635415,1.515259,1.256053,1.827996,0.005687
IGFALS,9.192586e-05,0.623219,0.729075,0.626061,0.849055,0.013334
VSIG4,9.569946e-05,0.637899,1.411417,1.194834,1.667331,0.013334
FGL1,1.184633e-04,0.625011,1.388001,1.180091,1.632593,0.014855


In [46]:
significant_proteins_discovery_fdr.shape

(28, 7)

In [16]:
significant_proteins_discovery_fdr = significant_proteins_discovery_fdr.reset_index()
significant_proteins_discovery_fdr = significant_proteins_discovery_fdr.rename(columns={"index": "protein"})
significant_proteins_discovery_fdr = significant_proteins_discovery_fdr.round(6)


In [17]:
significant_proteins_discovery_fdr

,protein,p_value,concordance_index,hazard_ratio,CI_lower,CI_upper,FDR_adjusted_pval
0,CILP2,0.000000,0.650983,0.591399,0.493154,0.709223,0.000055
1,REG1A,0.000001,0.629534,1.513684,1.288619,1.778114,0.000336
2,GRN,0.000001,0.650086,1.541034,1.306702,1.817437,0.000336
3,MRC1,0.000003,0.640314,1.529697,1.292062,1.811114,0.000891
4,IGFBP2,0.000005,0.636588,1.504184,1.269023,1.782994,0.001140
5,ADAM15,0.000007,0.637693,1.475305,1.254887,1.734510,0.001531
6,EPHB4,0.000032,0.635415,1.515259,1.256053,1.827996,0.005687
7,IGFALS,0.000092,0.623219,0.729075,0.626061,0.849055,0.013334
8,VSIG4,0.000096,0.637899,1.411417,1.194834,1.667331,0.013334
9,FGL1,0.000118,0.625011,1.388001,1.180091,1.632593,0.014855


In [19]:
discovery_after_fdr = significant_proteins_discovery_fdr["protein"].to_list()

In [75]:
discovery_after_fdr

['CILP2',
 'REG1A',
 'GRN',
 'MRC1',
 'IGFBP2',
 'ADAM15',
 'EPHB4',
 'IGFALS',
 'VSIG4',
 'FGL1',
 'SPON1',
 'APMAP',
 'PON1',
 'PTPRJ',
 'IGDCC4',
 'EXTL2',
 'SHBG',
 'IGF1',
 'APOF',
 'CTSB',
 'LRG1',
 'FKBP1A',
 'CNTN3',
 'GHR',
 'SECTM1',
 'CDH1',
 'NPC2',
 'FETUB',
 'ICOSLG',
 'FRMPD3']

In [18]:
#save results
significant_proteins_discovery_fdr.to_csv('significant_proteins_discovery_fdr_to_paper.csv', index=True)

### Replication

**Sort the DataFrame by p-values in decreasing order**

In [21]:
# Step 1: Sort the DataFrame by p-values in **decreasing order**
Replication = results_cv.sort_values(by='p_value', ascending=True)
Replication


,p_value,concordance_index,hazard_ratio,CI_lower,CI_upper
LRG1,0.000817,0.610778,1.327694,1.131062,1.558522
IGFBP2,0.041432,0.522480,1.190478,1.011314,1.401407
IGDCC4,0.041933,0.558064,0.833921,0.701695,0.991129
GRN,0.081882,0.547608,1.157725,0.987242,1.357663
REG1A,0.090360,0.530321,1.167745,0.980700,1.390515
EXTL2,0.123003,0.566702,0.869962,0.731771,1.034274
NBL1,0.142617,0.549312,1.136657,0.962993,1.341721
APOF,0.143122,0.565336,1.131175,0.969680,1.319580
FKBP1A,0.151181,0.548736,1.143688,0.958944,1.364057
GHR,0.155227,0.541942,0.884858,0.750315,1.043530


**Apply FDR correction (Benjamini-Hochberg method)**

In [35]:
# Step 2: Apply FDR correction (Benjamini-Hochberg method) and store in a new column
#Replication['FDR_adjusted_pval'] = multipletests(Replication['p_value'], method='fdr_bh')[1]


**Filter significant proteins(<0.05)**

In [41]:

significant_proteins_Replication = Replication[Replication['p_value'] <= 0.05]
significant_proteins_Replication = significant_proteins_Replication.reset_index()


In [43]:
significant_proteins_Replication.rename(columns={"index": "protein"}, inplace=True)
significant_proteins_Replication = significant_proteins_Replication.round(6)
significant_proteins_Replication


,protein,p_value,concordance_index,hazard_ratio,CI_lower,CI_upper
0,LRG1,0.000817,0.610778,1.327694,1.131062,1.558522
1,IGFBP2,0.041432,0.522480,1.190478,1.011314,1.401407
2,IGDCC4,0.041933,0.558064,0.833921,0.701695,0.991129


___

### Keep only Protein that replicate :

In [27]:
# Step 2: Filter both DataFrames to keep only common proteins
common_proteins = set(significant_proteins_discovery["protein"]).intersection(set(significant_proteins_Replication["protein"]))

dis_filtered = significant_proteins_discovery[significant_proteins_discovery["protein"].isin(common_proteins)]
rep_filtered = significant_proteins_Replication[significant_proteins_Replication["protein"].isin(common_proteins)]

NameError: name 'significant_proteins_discovery' is not defined

In [44]:
#save results
#significant_proteins_dis.to_csv('significant_proteins_discovery_fdr_to_paper.csv'', index=False)
significant_proteins_Replication.to_csv('significant_proteins_replication_fdr_to_paper.csv', index=True)

